In [1]:
import pandas as pd

In [2]:
gov_corruption_df = pd.read_csv('clean data/cpi_2019.csv')
generosity_df = pd.read_csv('clean data/global_generosity_score.csv')
human_freedom_df = pd.read_csv('clean data/hf_score.csv')
life_expectancy_df = pd.read_csv('clean data/lifeexp.csv')
education_df = pd.read_csv('clean data/clean_edu.csv')
gdp_df = pd.read_csv('clean data/Updated_GDP.csv')
happy_df = pd.read_csv('clean data/happy_score.csv')

In [3]:

gov_corruption_df = gov_corruption_df.drop('Unnamed: 0',axis=1)
generosity_df = generosity_df.drop('Unnamed: 0',axis=1)


In [4]:
clean_corrupt_df = gov_corruption_df.sort_values('Country')
clean_corrupt_df = clean_corrupt_df.reset_index(drop=True)
#clean_corrupt_df

In [5]:
gdp_df = gdp_df.rename(columns={'Country Name':'Country','Country Code':'Region','2019':'GDP'})
gdp_df = gdp_df[['Country','GDP']]


happy_df = happy_df.rename(columns={'Country or region':'Country','Score':'Happiness Score'})
#happy_df

In [6]:
clean_gen_df = generosity_df.rename(columns={'country':'Country','overallScore':'Generosity Score'})
clean_gen_df = clean_gen_df.sort_values('Country')
clean_gen_df = clean_gen_df.reset_index(drop=True)
#clean_gen_df

In [7]:
human_freedom_df = human_freedom_df.rename(columns={'countries':'Country','region':'Region','hf_score':'Human Freedom Score'})
#human_freedom_df

In [8]:
education_df = education_df.rename(columns={'Data Source':'Country','Rate':'Education'})
#education_df

In [9]:
merge_1 = pd.merge(clean_corrupt_df,clean_gen_df,how='outer',on='Country')
merge_2 = pd.merge(merge_1,human_freedom_df,how='outer',on='Country')
merge_3 = pd.merge(merge_2,life_expectancy_df,how='outer',on='Country')
merge_4 = pd.merge(merge_3,education_df,how='outer',on='Country')
merge_5 = pd.merge(merge_4,gdp_df,how='outer',on='Country')
final_data_df = pd.merge(merge_5,happy_df,how='outer',on='Country')


In [10]:
final_data_df = final_data_df[['Country','Region','Happiness Score','CPI score 2019','Generosity Score',
                               'Human Freedom Score','Life Expectancy','Education','GDP']]

final_data_df

,Country,Region,Happiness Score,CPI score 2019,Generosity Score,Human Freedom Score,Life Expectancy,Education,GDP
0,Afghanistan,AP,3.203,16.0,31.0,NaN,63.2,53.038544,507.103432
1,Albania,ECA,4.719,35.0,23.0,7.81,78.0,94.153337,5353.244856
2,Algeria,MENA,5.211,35.0,NaN,5.20,77.1,81.234412,3973.964072
3,Angola,SSA,NaN,26.0,NaN,5.48,63.1,NaN,2790.726615
4,Argentina,AME,6.086,45.0,28.0,7.05,76.6,89.302147,9912.281809
...,...,...,...,...,...,...,...,...,...
305,Ivory Coast,NaN,4.944,NaN,NaN,NaN,NaN,NaN,NaN
306,Congo (Brazzaville),NaN,4.812,NaN,NaN,NaN,NaN,NaN,NaN
307,Palestinian Territories,NaN,4.696,NaN,NaN,NaN,NaN,NaN,NaN
308,Congo (Kinshasa),NaN,4.418,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
final_data_df.dropna(axis=0,how='any')

,Country,Region,Happiness Score,CPI score 2019,Generosity Score,Human Freedom Score,Life Expectancy,Education,GDP
1,Albania,ECA,4.719,35.0,23.0,7.81,78.0,94.153337,5353.244856
4,Argentina,AME,6.086,45.0,28.0,7.05,76.6,89.302147,9912.281809
5,Armenia,ECA,4.559,42.0,21.0,7.69,76.0,90.511539,4622.733493
7,Austria,WE/EU,7.246,77.0,45.0,8.45,81.6,97.130831,50137.662776
8,Azerbaijan,ECA,5.208,30.0,24.0,6.29,71.4,86.941317,4793.587020
...,...,...,...,...,...,...,...,...,...
167,Uganda,SSA,4.189,28.0,38.0,6.58,66.7,28.755817,794.341078
168,Ukraine,ECA,4.332,30.0,24.0,6.45,73.0,96.159267,3659.031312
169,United Arab Emirates,MENA,6.825,71.0,45.0,6.17,76.1,78.312130,43103.323058
178,Zambia,SSA,4.107,34.0,39.0,6.71,62.5,54.768871,1305.063254
